In [142]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [143]:
cargadores=pd.read_excel(r"C:\Users\Emmanuel\OneDrive - Instituto Politecnico Nacional\GITHUB\charger-stations-data-cleaned\CHARGER STATIONS\cargadores_electromaps.xlsx")
cargadores.head()

,Direccion,CP,Tipo,Nombre_App,Alcaldia,Tipo_conector,Potencia_kW,Marca
0,115 Avenida Paseo del Río 04360,4360.0,Centro Comercial,Walmart Miguel Angel de Quevedo,Álvaro Obregón,TYPE 1 (SAE J1772),3.7,Walmart
1,227 Avenida Miguel Ángel de Quevedo 04360,4360.0,Centro Comercial,Oasis Coyoacan,Coyoacán,TYPE 1 (SAE J1772),3.7,NaN
2,227 Avenida Miguel Ángel de Quevedo 04360,4360.0,Centro Comercial,Oasis Coyoacan,Coyoacán,TYPE 1 (SAE J1772),3.7,NaN
3,85 Vito Alessio Robles 01030,1030.0,Estacionamiento,Centro Escolar Cedros,Álvaro Obregón,TYPE 1 (SAE J1772),3.7,NaN
4,85 Vito Alessio Robles 01030,1030.0,Estacionamiento,Centro Escolar Cedros,Álvaro Obregón,TYPE 1 (SAE J1772),3.7,NaN


## Limpieza de datos

In [144]:
cargadores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Direccion      161 non-null    object 
 1   CP             158 non-null    float64
 2   Tipo           160 non-null    object 
 3   Nombre_App     162 non-null    object 
 4   Alcaldia       162 non-null    object 
 5   Tipo_conector  162 non-null    object 
 6   Potencia_kW    138 non-null    float64
 7   Marca          46 non-null     object 
dtypes: float64(2), object(6)
memory usage: 10.3+ KB


## Sustitución de valores nulos
Arreglando los valores nulos de las columnas que tienen strings y el CP, ya que como no se conoce, se colocará un cero

In [145]:
cargadores[["Tipo","Marca","Direccion"]]=cargadores[["Tipo","Marca","Direccion"]].fillna("Desconocido")
cargadores.CP.fillna(0,inplace=True)
cargadores.CP=cargadores.CP.astype("int64")
cargadores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Direccion      162 non-null    object 
 1   CP             162 non-null    int64  
 2   Tipo           162 non-null    object 
 3   Nombre_App     162 non-null    object 
 4   Alcaldia       162 non-null    object 
 5   Tipo_conector  162 non-null    object 
 6   Potencia_kW    138 non-null    float64
 7   Marca          162 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 10.3+ KB


Para la potencia es más complicado, ya que no se puede poner una potencia de 0 kW, por lo que se procederá a revisar qué tipo de conector es, y se usará la potencia que mayormente sea instalada para ese tipo de conector

In [146]:
cargadores.Tipo_conector.value_counts()

TYPE 1 (SAE J1772)          66
Tesla Dest.Charger          43
Tesla Supercharger          10
TYPE 2                       9
CCS2                         8
NEMA 5-15 (US Plug)          6
CHAdeMO                      5
NEMA 14-50                   4
CCS1                         3
\nTYPE 1 (SAE J1772)         2
NEMA 5-20 (US Plug)          2
 TYPE 1 (SAE J1772)          1
Type J (CH, LI, RW Plug)     1
 CHAdeMO                     1
TYPE 1 (SAE J1772)           1
Name: Tipo_conector, dtype: int64

Se observa que se tiene faltas de ortografía y espacios en los extremos, se procede a arreglarlo

In [147]:
cargadores.Tipo_conector.replace("\nTYPE 1 (SAE J1772)","TYPE 1 (SAE J1772)",inplace=True)
cargadores.Tipo_conector.value_counts()

TYPE 1 (SAE J1772)          68
Tesla Dest.Charger          43
Tesla Supercharger          10
TYPE 2                       9
CCS2                         8
NEMA 5-15 (US Plug)          6
CHAdeMO                      5
NEMA 14-50                   4
CCS1                         3
NEMA 5-20 (US Plug)          2
 TYPE 1 (SAE J1772)          1
Type J (CH, LI, RW Plug)     1
 CHAdeMO                     1
TYPE 1 (SAE J1772)           1
Name: Tipo_conector, dtype: int64

Debido a que los espacios en los extremos es posible que se encuentren en todas las columnas con cadenas, se procede a quitarle esos espacios a todas las columnas

In [148]:
cargadores.Tipo_conector=cargadores.Tipo_conector.str.strip()
cargadores.Direccion=cargadores.Direccion.str.strip()
cargadores.Tipo=cargadores.Tipo.str.strip()
cargadores.Nombre_App=cargadores.Nombre_App.str.strip()
cargadores.Marca=cargadores.Marca.str.strip()
cargadores.Alcaldia=cargadores.Alcaldia.str.strip()

Se procede a revisar la moda en la potencia para cada tipo de conector

In [149]:
len(cargadores[cargadores.Tipo_conector == "NEMA 5-20 (US Plug)"].Potencia_kW.value_counts())

0

In [150]:
cargadores.loc[((pd.isnull(cargadores.Potencia_kW)==True)  & (cargadores.Tipo_conector == "NEMA 5-20 (US Plug)")) ,"Potencia_kW"] = 0

In [151]:
for conector in pd.unique(cargadores.Tipo_conector).tolist():
    if len(cargadores[cargadores.Tipo_conector == conector].Potencia_kW.value_counts()) == 0:
        print(f"El conector {conector} no tiene potencia de referencia, por lo tanto, se ajustará su valor a 0")
        cargadores.loc[((pd.isnull(cargadores.Potencia_kW)==True)  & (cargadores.Tipo_conector == conector)),"Potencia_kW"] = 0
    else:
        moda_conector=float(cargadores.query("Tipo_conector == @conector").Potencia_kW.value_counts().index[0])
        print(f"Conector: {conector}, moda de potencia: {moda_conector} kW")
        cargadores.loc[((pd.isnull(cargadores.Potencia_kW)==True)  & (cargadores.Tipo_conector == conector)),"Potencia_kW"] = moda_conector
            

Conector: TYPE 1 (SAE J1772), moda de potencia: 6.0 kW
Conector: CCS2, moda de potencia: 50.0 kW
Conector: TYPE 2, moda de potencia: 22.0 kW
Conector: CCS1, moda de potencia: 50.0 kW
Conector: Tesla Dest.Charger, moda de potencia: 11.0 kW
Conector: Type J (CH, LI, RW Plug), moda de potencia: 7.4 kW
Conector: NEMA 5-20 (US Plug), moda de potencia: 0.0 kW
Conector: NEMA 5-15 (US Plug), moda de potencia: 2.0 kW
Conector: CHAdeMO, moda de potencia: 22.0 kW
Conector: NEMA 14-50, moda de potencia: 7.0 kW
Conector: Tesla Supercharger, moda de potencia: 120.0 kW


In [152]:
cargadores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Direccion      162 non-null    object 
 1   CP             162 non-null    int64  
 2   Tipo           162 non-null    object 
 3   Nombre_App     162 non-null    object 
 4   Alcaldia       162 non-null    object 
 5   Tipo_conector  162 non-null    object 
 6   Potencia_kW    162 non-null    float64
 7   Marca          162 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 10.3+ KB


In [153]:
cargadores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Direccion      162 non-null    object 
 1   CP             162 non-null    int64  
 2   Tipo           162 non-null    object 
 3   Nombre_App     162 non-null    object 
 4   Alcaldia       162 non-null    object 
 5   Tipo_conector  162 non-null    object 
 6   Potencia_kW    162 non-null    float64
 7   Marca          162 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 10.3+ KB
